Uploading an image with graphical annotations stored in a CSV file
======================

We'll be using standard python tools to parse CSV and create an XML document describing cell nuclei for BisQue

Make sure you have bisque api installed:
> pip install bisque-api

In [ ]:
import os
import csv
from datetime import datetime
try:
    from lxml import etree
except ImportError:
    import xml.etree.ElementTree as etree

Include BisQue API

In [ ]:
from bqapi import BQSession, BQCommError
from bqapi.util import save_blob, localpath2url

define some paths

In [ ]:
path = './files_annot'
path_img = os.path.join(path, 'BisQue_CombinedSubtractions.lsm')
path_csv = os.path.join(path, 'BisQue_CombinedSubtractions.csv')

Parse CSV file and load nuclei positions
------------------------------------------

We'll create a list of XYZT coordinates with confidence

In [ ]:
#x, y, z, t, confidence
coords = []
with open(path_csv, 'rb') as csvfile:
    reader = csv.reader(csvfile)
    h = reader.next()
    for r in reader:
        c = (r[0], r[1], r[2], r[4])
        print c
        coords.append(c)

Initiaize authenticated session
--------------

Initialize a BisQue session using simple user credentials

In [ ]:
root = 'http://bisque.ece.ucsb.edu'
user = 'my_user'
pswd = 'my_pass'

session = BQSession().init_local(user, pswd,  bisque_root=root, create_mex=False)

Create XML image descriptor
---------------------------

We'll provide a suggested path in the remote user's directory 

In [ ]:
path_on_bisque = 'demo/nuclei_%s/%s'%(datetime.now().strftime('%Y%m%dT%H%M%S'), os.path.basename(path_img))
resource = etree.Element ('image', name=path_on_bisque)
print etree.tostring(resource, pretty_print=True)

Upload the image
-----------------

In [ ]:
# use import service to /import/transfer activating import service
r = etree.XML(session.postblob(path_img, xml=resource)).find('./') 

if r is None or r.get('uri') is None:
    print 'Upload failed'
else:
    print 'Uploaded ID: %s, URL: %s\n'%(r.get('resource_uniq'), r.get('uri'))
    print etree.tostring(r, pretty_print=True)

Add graphical annotations
------------------------------

We'll create point annotaions as an XML attached to the image we just uploaded into BisQue

In [ ]:
g = etree.SubElement (r, 'gobject', type='My nuclei')
for c in coords:
    p = etree.SubElement (g, 'point')
    etree.SubElement (p, 'vertex', x=c[0], y=c[1], z=c[2])
    etree.SubElement (p, 'tag', name='confidence', value=c[3])

print etree.tostring(r, pretty_print=True)

Save graphical annotations to the system
------------------------------------------

After storing all annotations become searchable 

In [ ]:
url = session.service_url('data_service')
r = session.postxml(url, r)

if r is None or r.get('uri') is None:
    print 'Adding annotations failed'
else:
    print 'Image ID: %s, URL: %s'%(r.get('resource_uniq'), r.get('uri'))